In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

In [2]:
data = pd.read_csv('lenta-ru-news.csv')

C:\Users\seryy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.shape

(800975, 6)

In [4]:
data = data[:500000]

In [5]:
data.head(3)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17


In [6]:
#print(data['tags'].unique(), len(data['tags'].unique()))

In [7]:
print(data['topic'].unique(), len(data['topic'].unique()))

['Библиотека' 'Россия' 'Мир' 'Экономика' 'Интернет и СМИ' 'Спорт'
 'Культура' 'Из жизни' 'Силовые структуры' 'Наука и техника' 'Бывший СССР'
 nan 'Дом' 'Сочи'] 14


In [8]:
#Выборка данных

In [9]:
topics = ['Россия', 'Спорт', 'Мир', 'Наука и техника', 'Экономика']
news_in_cat_count = 2000

In [10]:
df_res = pd.DataFrame()

for topic in tqdm(topics):
    df_topic = data[data['topic'] == topic][:news_in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
df_res.shape

(10000, 6)

In [12]:
df_res['text'][300]

'В конце прошлой недели сотрудники Государственного таможенного комитета  и Федеральной службы безопасности РФ провели антипиратскую акцию. Однако если прежде правоохранительные органы конфисковывали пиратскую продукцию и задерживали ее распространителей, то в этот раз акция была направлена непосредственно против производителей пиратских компакт-дисков, -сообщает агентство "Алгоритм". Оперативно-розыскные действия проводились одновременно в подмосковном Зеленограде и на административной российско-белорусской границе, где и была задержана машина с 13 тысячами произведенных в Зеленограде компакт-дисков (их конечным местом назначения была Польша). После задержания машины сотрудники ГТК и ФСБ провели обыск и задержания на подпольном заводе в Зеленограде, который располагался в арендованном у режимного предприятия "Компонент" (занимается производством микросхем для ВПК) помещении. Хозяином завода оказалась фирма "Диск-пресс МСК". По данным правоохранительных органов, "Диск-пресс МСК" принад

In [13]:
#Предобработка

In [14]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

In [15]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in df_res['text']]

In [16]:
prep_text[300]

'в конце прошлой недели сотрудники государственного таможенного комитета и федеральной службы безопасности рф провели антипиратскую акцию однако если прежде правоохранительные органы конфисковывали пиратскую продукцию и задерживали ее распространителей то в этот раз акция была направлена непосредственно против производителей пиратских компакт дисков сообщает агентство алгоритм оперативно розыскные действия проводились одновременно в подмосковном зеленограде и на административной российско белорусской границе где и была задержана машина с тысячами произведенных в зеленограде компакт дисков их конечным местом назначения была польша после задержания машины сотрудники гтк и фсб провели обыск и задержания на подпольном заводе в зеленограде который располагался в арендованном у режимного предприятия компонент занимается производством микросхем для впк помещении хозяином завода оказалась фирма диск пресс мск по данным правоохранительных органов диск пресс мск принадлежит гражданину болгарии —

In [17]:
#df_res['text'][300]

In [18]:
df_res['text_prep'] = prep_text

In [19]:
df_res.head(3)

,url,title,text,topic,tags,date,text_prep
0,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,как стало известно агентству ассошиэйтед пресс...
1,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,в зале игровых автоматов в третьем ярусе подзе...
2,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,япония приняла решение разморозить кредиты рос...


In [20]:
df_res.shape

(10000, 7)

In [21]:
#Стемминг

In [22]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian")

In [23]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seryy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
from nltk import word_tokenize
text = df_res['text_prep'][300]
word_tokenize(text)
#print(russian_stopwords)

['в',
 'конце',
 'прошлой',
 'недели',
 'сотрудники',
 'государственного',
 'таможенного',
 'комитета',
 'и',
 'федеральной',
 'службы',
 'безопасности',
 'рф',
 'провели',
 'антипиратскую',
 'акцию',
 'однако',
 'если',
 'прежде',
 'правоохранительные',
 'органы',
 'конфисковывали',
 'пиратскую',
 'продукцию',
 'и',
 'задерживали',
 'ее',
 'распространителей',
 'то',
 'в',
 'этот',
 'раз',
 'акция',
 'была',
 'направлена',
 'непосредственно',
 'против',
 'производителей',
 'пиратских',
 'компакт',
 'дисков',
 'сообщает',
 'агентство',
 'алгоритм',
 'оперативно',
 'розыскные',
 'действия',
 'проводились',
 'одновременно',
 'в',
 'подмосковном',
 'зеленограде',
 'и',
 'на',
 'административной',
 'российско',
 'белорусской',
 'границе',
 'где',
 'и',
 'была',
 'задержана',
 'машина',
 'с',
 'тысячами',
 'произведенных',
 'в',
 'зеленограде',
 'компакт',
 'дисков',
 'их',
 'конечным',
 'местом',
 'назначения',
 'была',
 'польша',
 'после',
 'задержания',
 'машины',
 'сотрудники',
 'гтк',


In [25]:
stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list

  0%|          | 0/10000 [00:00<?, ?it/s]

In [26]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [27]:
df_res.head(1)

,url,title,text,topic,tags,date,text_prep,text_stem
0,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,как стало известно агентству ассошиэйтед пресс...,стал известн агентств ассошиэйтед пресс экипаж...


In [28]:
sw_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_res['text_sw'] = sw_texts_list

  0%|          | 0/10000 [00:00<?, ?it/s]

In [29]:
df_res['text_sw'][300]

'конце прошлой недели сотрудники государственного таможенного комитета федеральной службы безопасности рф провели антипиратскую акцию однако прежде правоохранительные органы конфисковывали пиратскую продукцию задерживали распространителей акция направлена непосредственно против производителей пиратских компакт дисков сообщает агентство алгоритм оперативно розыскные действия проводились одновременно подмосковном зеленограде административной российско белорусской границе задержана машина тысячами произведенных зеленограде компакт дисков конечным местом назначения польша задержания машины сотрудники гтк фсб провели обыск задержания подпольном заводе зеленограде который располагался арендованном режимного предприятия компонент занимается производством микросхем впк помещении хозяином завода оказалась фирма диск пресс мск данным правоохранительных органов диск пресс мск принадлежит гражданину болгарии — одному компаньонов арестованного году филиппа стайкова кличка — болгарин используемая пи

In [30]:
df_res.to_csv('lenta_stemmed.csv')

In [31]:
df_res['text_stem'][300]

'конц прошл недел сотрудник государствен таможен комитет федеральн служб безопасн рф провел антипиратск акц однак прежд правоохранительн орга конфисковыва пиратск продукц задержива распространител акц направл непосредствен прот производител пиратск компакт диск сообща агентств алгоритм оперативн розыскн действ провод одновремен подмосковн зеленоград административн российск белорусск границ задержа машин тысяч произведен зеленоград компакт диск конечн мест назначен польш задержан машин сотрудник гтк фсб провел обыск задержан подпольн завод зеленоград котор располага арендова режимн предприят компонент занима производств микросх впк помещен хозяин завод оказа фирм диск пресс мск дан правоохранительн орган диск пресс мск принадлеж гражданин болгар — одн компаньон арестова год филипп стайков кличк — болгарин используем пиратск фирм производствен лин приобрет чех привез росс вид кузнечн пресс стоимост оборудован состав окол миллион доллар сша сутк лин напечата тысяч компакт диск котор прода

In [32]:
df_res.head(1)

,url,title,text,topic,tags,date,text_prep,text_stem,text_sw
0,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,как стало известно агентству ассошиэйтед пресс...,стал известн агентств ассошиэйтед пресс экипаж...,стало известно агентству ассошиэйтед пресс эки...


In [33]:
df_res.shape

(10000, 9)

In [34]:
#Лемматизация

In [35]:
lemm_texts_list = []
for text in tqdm(df_res['text_stem']):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
    text = " ".join(tokens)
    lemm_texts_list.append(text)

df_res['text_lemm'] = lemm_texts_list

  0%|          | 0/10000 [00:00<?, ?it/s]

In [36]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [37]:
df_res.to_csv('lemm.csv')

In [38]:
#df_res = pd.read_csv('lemm.csv', encoding = 'utf-8')

In [39]:
df_res.head(1)

,url,title,text,topic,tags,date,text_prep,text_stem,text_sw,text_lemm
0,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,как стало известно агентству ассошиэйтед пресс...,стал известн агентств ассошиэйтед пресс экипаж...,стало известно агентству ассошиэйтед пресс эки...,становиться известна агентство ассошиэйтед пре...


In [40]:
df_res.shape

(10000, 10)

In [41]:
df_res['text_lemm'][300]

'конц прошлый недел сотрудник государственный таможня комитет федеральна служба безопасна рф проводить антипиратск акца однак прежды правоохранительн орг конфисковыв пиратск продукца задержив распространител акца направл непосредственный прот производител пиратск компакт диск сообща агентство алгоритм оперативна розыскна действо провод одновремя подмосковн зеленоград административна российск белорусск граница задерж машина тысяча производить зеленоград компакт диск конечный место назначать польша задерживать машина сотрудник гтк фсб проводить обыск задерживать подпольн завод зеленоград котор располага арендова режимн предприять компонент заним производство микросх впк помещать хозяин завод оказ фирма диск пресс мск давать правоохранительн орган диск пресс мск принадлеж гражданин болгарин  —  одн компаньон арестов год филипп стайков клички  —  болгарин использовать пиратск фирма производственный лин приобрет чех привозить росс вид кузнечный пресс стоимост оборудовать состав окол миллион

In [42]:
X = df_res['text_lemm']
y = df_res['topic']

In [43]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=None, train_size=None, random_state = 0)

In [44]:
my_tags = df_res['topic'].unique()
my_tags

array(['Россия', 'Спорт', 'Мир', 'Наука и техника', 'Экономика'],
      dtype=object)

In [45]:
#Naive Bayes Classifier

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_accuracies = cross_val_score(estimator=nb, X=X_train, y=y_train, cv=cv)

In [49]:
print(all_accuracies.mean())

0.8761333333333333


In [50]:
#%%time
#nb.fit(X_train, y_train)

In [51]:
#nb.score(X_test, y_test)

In [52]:
#%%time
#from sklearn.metrics import classification_report
#y_pred = nb.predict(X_test)

In [53]:
#from sklearn.metrics import accuracy_score

#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred, target_names=my_tags))

In [54]:
#Support Vector Machine

In [55]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_accuracies = cross_val_score(estimator=sgd, X=X_train, y=y_train, cv=cv)

In [57]:
print(all_accuracies.mean())

0.8908000000000001


In [58]:
#%%time
#sgd.fit(X_train, y_train)

In [59]:
#sgd.score(X_test, y_test)

In [60]:
#%%time
#y_pred = sgd.predict(X_test)

In [61]:
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [62]:
#Logistic Regression

In [63]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=1000)),
               ])

In [64]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_accuracies = cross_val_score(estimator=logreg, X=X_train, y=y_train, cv=cv)

In [65]:
print(all_accuracies.mean())

0.9018666666666666


In [66]:
#%%time
#logreg.fit(X_train, y_train)

In [67]:
#logreg.score(X_test, y_test)

In [68]:
#%%time
#y_pred = logreg.predict(X_test)

In [69]:
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [70]:
#k-Nearest Neighbors

In [71]:
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [72]:
kNN = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier(n_neighbors = 7)),
               ])

In [73]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_accuracies = cross_val_score(estimator=kNN, X=X_train, y=y_train, cv=cv)

In [74]:
print(all_accuracies.mean())

0.8425333333333332


In [75]:
#kNN.fit(X_train, y_train)

In [76]:
#kNN.score(X_test, y_test)

In [77]:
#%%time
#y_pred = kNN.predict(X_test)

In [78]:
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [79]:
#Decision Trees

In [80]:
from sklearn.tree import DecisionTreeClassifier

In [81]:
dtc = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier()),
               ])

In [82]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_accuracies = cross_val_score(estimator=dtc, X=X_train, y=y_train, cv=cv)

In [83]:
print(all_accuracies.mean())

0.7206666666666666


In [84]:
#dtc.fit(X_train, y_train)

In [85]:
#dtc.score(X_test, y_test)

In [86]:
#%%time
#y_pred = dtc.predict(X_test)

In [87]:
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))